In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'    

if tf.test.gpu_device_name():
    print('GPU found', tf.test.gpu_device_name())
else:
    print("No GPU found")

No GPU found


In [ ]:
model = VGG16()
model.summary()

In [2]:
from vgg import VGG

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y_true = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

model = VGG(x, keep_prob, 10)


W0212 11:15:36.514166 140431839504192 deprecation_wrapper.py:119] From /home/harry/DeepLearning/StyleTransfer_VGG/tf/vgg.py:41: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0212 11:15:36.515261 140431839504192 deprecation_wrapper.py:119] From /home/harry/DeepLearning/StyleTransfer_VGG/tf/vgg.py:43: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0212 11:15:36.537331 140431839504192 deprecation_wrapper.py:119] From /home/harry/DeepLearning/StyleTransfer_VGG/tf/vgg.py:86: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0212 11:15:36.649642 140431839504192 deprecation.py:506] From /home/harry/DeepLearning/StyleTransfer_VGG/tf/vgg.py:98: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
